In [88]:
# given the location of the tm.json metadata file, download the tiles from google storage
# mark egge, 23 june 2016, CREATE Lab

# user defined parameters:
tmDotJsonUrl = 'http://storage.googleapis.com/earthengine-timelapse/evi_animation_test_megge_001/tm.json'

offset = 4 # time machine uses only every 4th tile. Set to zero for normal tiling applications

levelLimit = 7 # -1 be default. only downloads first 7 zoom levels

# script code:
import json
import urllib, urllib2
import os
import string

baseUrl = tmDotJsonUrl[:string.rfind(tmDotJsonUrl, '/')]

# list of one-off files:
files = ['ajax_includes.js', 'tm.json', '1068x600/r.json'] # misc non-tile files to download

# get tile directory from tm.json
response = urllib2.urlopen(tmDotJsonUrl)
tmDotJsonText = response.read()
tmDotJson = json.loads(tmDotJsonText)

tileDirectory = tmDotJson['datasets'][0]['name']

# get levels info from r.json
response = urllib2.urlopen(baseUrl + '/' + tileDirectory + '/' + 'r.json')
rDotJsonText = response.read()
rDotJson = json.loads(rDotJsonText)

levels = rDotJson["level_info"]

# prune levels, if need be
if levelLimit >= 0 and levelLimit < len(levels):
    levels = levels[0:levelLimit]

file = urllib.URLopener()

def makeDirectory(directory):
    if not os.path.exists(directory):
        try:
            os.makedirs(directory)    
        except:
            print 'error making directory: ' + directory
    
def downloadFile(filePath):
    if filePath[0] == '/':
        filePath = filePath[1:] # strip leading slash, if any
        
    parts = string.split(filePath, '/')
    if len(parts) > 1:
        makeDirectory(filePath[:-len(parts[-1])])
        
    fileUrl = baseUrl + '/' + filePath
    
    if not os.path.isfile(filePath):
        try:
            file.retrieve(fileUrl, filePath)
        except IOError:
            print 'error downloading ' + fileUrl + ' to ' + filePath

# download the one-offs
for f in files:
    downloadFile(f) 

# download tiles
for x in range(len(levels)): 
    column = 0
    while column < levels[x]['cols']:
        row = 0
        #print rowcol[x]['cols']
        while row < levels[x]['rows']:
            fileName = tileDirectory + '/' + str(x) + '/' + str(row) + '/' + str(column) + '.mp4'
            downloadFile(fileName)
            
            row = row + offset
        column = column + offset
        
print 'tiles downloaded'

tiles downloaded
